In [1]:
### USE ONLY THESE PACKAGES ###
import os
import csv
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

### Upload the lyrics_dataset.zip in the colab's folder first !

In [ ]:
!unzip lyrics_dataset.zip

### Do not change this seed number

In [2]:
np.random.seed(1)
torch.manual_seed(1)

In [3]:
## Use below code to train your model with all lyrics
# lyrics = list()
# for txt_file in os.listdir('./lyrics_dataset'):
#     if txt_file[0] != '.':
#         target_txt = os.path.join('./lyrics_dataset', txt_file)
#         f = open(target_txt, 'r')
#         curr_lyrics = f.readlines()
#         for i in range(len(curr_lyrics)):
#             curr_lyrics[i] = curr_lyrics[i].lower()
#         curr_lyrics = list(set(curr_lyrics))
#         lyrics += curr_lyrics
# print(len(lyrics))

## Use below code to train your model with specific artist's lyrics
target_txt = './lyrics_dataset/lady-gaga.txt'
f = open(target_txt, 'r')
lyrics = f.readlines()
for i in range(len(lyrics)):
    lyrics[i] = lyrics[i].lower()
lyrics = list(set(lyrics))

# #"lyrics" is a list of strings.
#for l in lyrics:
#     print(l)

In [4]:
import re
import collections
import numpy as np

In [29]:
class lyricsDataset(Dataset): # 12 points
    def __init__(self, lyrics, window_len):
        self.lyrics = lyrics
        self.window_len = window_len
        # window len means, that, if i-th word is target
        # you would consider i-window_len ~ i+window_len words as context words

        ##--------------write below-------------## <--- do not erase this afterwards
        self.trimmed_lyrics = list()    
        self.vocabulary = list()
        self.frequency = dict()
        # your code in here
        for i in range(len(lyrics)):
            filtered_string = re.sub("[^0-9a-z]+", " ", lyrics[i])
            string2list = filtered_string.split() 
            self.trimmed_lyrics.append(string2list)
        
        merged_list = list()
        for i in self.trimmed_lyrics:
            merged_list.extend(i)
        
        list_set = set(merged_list) 
        self.vocabulary = list(list_set)
        
        self.frequency = collections.Counter(merged_list)
        ##--------------write above-------------## <--- do not erase this   
        
        ### MUST TO-DO 1. (+3) -> self.trimmed_lyrics
        # for strings in self.lyrics, 
        # (+2) write a code to eliminate every character except for alphabet, number, and space (" ")
        ### for example, - + ? ! ' " [ ] ( ) <- char like this should be excluded. 
        # (+1) after that, split each string with respect to the space.
        ### If proprocessed string is "hello hello hello", 
        ### a list ['hello', 'hello', 'hello'] should be generated.
        ### Then, put that list into the self.trimmed_lyrics.
        ### self.trimmed_lyrics needs to be a LIST which has LIST as element.

        ### MUST TO-DO 2. (+2) -> self.vocabulary
        # (+2) Put all words(string) in self.trimmed_lyrics to the self.vocabulary. 
        ### In self.vocabulary, each word needs to be unique.
        ### which means, this list (self.vocabulary) should not have duplicated elements. 
        ### self.vocabulary needs to be a LIST which contains unique words in self.trimmed_lyrics
        ### If your code contains duplicated words, you would not get the point.
        ### If there is a word that are neglected from self.trimmed_lyrics, you would not get the point.

        ### MUST TO-DO 3. (+2) -> self.frequency
        # (+2) In self.frequency, Record how many times each word in self.vocabulary 
        ###                              appears in self.trimmed_lyrics.
        ### For example, if "love" appears 100 times in self.trimmed_lyrics,
        ### it should be: self.frequency["love"] = 100

    def __len__(self):
        # DO NOT TOUCH BELOW. JUST USE BELOW CODE FOR YOUR __len__ 
        return len(self.trimmed_lyrics)
        # DO NOT TOUCH ABOVE. JUST USE ABOVE CODE FOR YOUR __len__ 

    def __getitem__(self, idx):
        ### MUST TO-DO 4. (+5) --> sample
        ##--------------write below-------------##
        sample = dict()
        sample['pairs'] = list()
        # your code here
        target_idx = np.random.randint(0,len(self.trimmed_lyrics[idx]))
        print(idx)
        print(target_idx)
        
        if len(self.trimmed_lyrics[idx]) == 1:
            sample['pairs'].append((self.vocabulary.index(self.trimmed_lyrics[idx][target_idx]),self.vocabulary.index(self.trimmed_lyrics[idx][target_idx])))
        
        else:
            for i in range(-self.window_len,self.window_len+1):
                context_idx = target_idx + i 
                if context_idx >= 0 and context_idx < len(self.trimmed_lyrics[idx]) and context_idx != target_idx:
                    sample['pairs'].append((self.vocabulary.index(self.trimmed_lyrics[idx][target_idx]),self.vocabulary.index(self.trimmed_lyrics[idx][context_idx])))
        
        return sample
        ##--------------write above-------------##
        # Bring out one list, from self.trimmed_lyrics. (i.e., self.trimmed_lyrics[idx])
        # Then, the list should contain the splited sentence in each lyric.
        ### i.e., self.trimmed_lyrics[idx] would look like ['hey', 'nice', 'to', 'meet', 'you']
        # (+1) randomly select the target word from self.trimmed_lyrics[idx]
        # (+4) based on selected target word, and self.window_len,
        #      generate a tuple of (target word index, context word index),
        #      and add that tuple into sample['pairs']
        ### if 'nice' is randomly chosen as the target element, and self.window_len = 1
        ### if self.vocabulary[100] == 'hey', 
        ###    self.vocabulary[500] == 'nice',
        ###    self.vocabulary[300] == 'to'
        ### sample['pairs'] should be [(500, 300), (500, 100)]

In [30]:
### DO NOT TOUCH BELOW. JUST USE THESE LINES.
### PENALTY (-5) CAN BE APPLIED IF YOUR CODE DOES NOT WORK FOR VARIOUS VALUES OF WINDOW_LEN
dataset = lyricsDataset(lyrics, 3)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
### DO NOT TOUCH ABOVE. JUST USE THESE LINES.

In [15]:
class Word2Vec(nn.Module): # 15 points
    def __init__(self, num_vocabs, embed_dim = 300): ## do not change 'embed_dim' value.
        ### MUST TO-DO 5 : (+5)
        ##--------------write below-------------##
        # your code here
        ##--------------write above-------------##
        # Define a model which can map word's integer index value to word embedding.
        self.num_vocabs = num_vocabs
        self.embed_dim = embed_dim
        super(Word2Vec, self).__init__()
        self.linear = torch.nn.Linear(in_features = self.num_vocabs, out_features = self.embed_dim)
        
    def forward(self, pairs):
        '''
        input : pairs - one tuple of (target word index, context word index)
        '''
        ### MUST TO DO 6 : (+10)
        ##--------------write below-------------##
        # your code here
        ##--------------write above-------------##
        # return the embedding of target word and context word.
        # i.e., return target_embed, context_embed
        target_onehot = torch.nn.functional.one_hot(pairs[0], num_classes=self.num_vocabs).float()
        context_onehot = torch.nn.functional.one_hot(pairs[1], num_classes=self.num_vocabs).float()
        target_embed = self.linear(target_onehot)
        context_embed = self.linear(context_onehot)
        
        return target_embed, context_embed

In [16]:
### DO NOT TOUCH BELOW. JUST USE THESE LINES. MAKE YOUR CODE WORK WITH THESE LINES
model = Word2Vec(len(dataset.vocabulary))
optimizer = torch.optim.AdamW(model.parameters())
device = 'cuda'
### DO NOT TOUCH ABOVE. JUST USE THESE LINES. MAKE YOUR CODE WORK WITH THESE LINES
### USE THIS GIVEN OPTIMIZER.

In [ ]:
len_prev_sample = len(prev_sample['pairs'])
len_sample = len(sample['pairs'])
first_target = torch.zeros(len_prev_sample+len_sample, dtype=torch.long)
second_target = torch.zeros(len_prev_sample+len_sample, dtype=torch.long)

#tuples of first target word 
first_tuples = list()
for i in range(len_sample):
    first_tuples.append(sample['pairs'][i])
    first_target[i] = 1 
    
for i in range(len_prev_sample):
    first_tuples.append((sample['pairs'][0][0],prev_sample['pairs'][i][1]))
    
#tuples of second target word 
second_tuples = list()
for i in range(len_prev_sample):
    second_tuples.append(prev_sample['pairs'][i])
    second_target[i] = 1 
    
for i in range(len_sample):
    second_tuples.append((prev_sample['pairs'][0][0],sample['pairs'][i][1]))
    

In [ ]:
first_predict_logit = torch.zeros(len_prev_sample+len_sample)
second_predict_logit = torch.zeros(len_prev_sample+len_sample)

for i in range(len(first_tuples)):
        target_embed, context_embed = model((first_tuples[i][0],first_tuples[i][1]))
        inner_product = torch.dot(torch.flatten(target_embed),torch.flatten(context_embed))
        first_predict_logit[i] = inner_product

m = nn.Softmax(dim=0)
result_first = m(first_predict_logit)
        
loss_first = -torch.sum(first_target * torch.log(result_first))


for i in range(len(second_tuples)):
        target_embed, context_embed = model((second_tuples[i][0],second_tuples[i][1]))
        inner_product = torch.dot(torch.flatten(target_embed),torch.flatten(context_embed))
        second_predict_logit[i] = inner_product
        
result_second = m(second_predict_logit)
        
loss_second = -torch.sum(second_target * torch.log(result_second))
        


In [ ]:
total_loss = loss_first + loss_second

In [ ]:
total_loss.backward()
optimizer.step()

In [17]:
def train(model, optimizer, sample_1, sample_2): # 13 points
    ### MUST TO DO 6 : (+12)
    ##--------------write below-------------##
    model.train()
    model = model.to(device)
    optimizer.zero_grad()
    
    first_target = torch.zeros(len(sample_2['pairs'])+len(sample_1['pairs']), dtype=torch.long).to(device)
    second_target = torch.zeros(len(sample_2['pairs'])+len(sample_1['pairs']), dtype=torch.long).to(device)
    
    #tuples of first target word 
    first_tuples = torch.LongTensor(len(sample_2['pairs'])+len(sample_1['pairs']),2).to(device)
    second_tuples = torch.LongTensor(len(sample_2['pairs'])+len(sample_1['pairs']),2).to(device)
    
    for i in range(len(sample_1['pairs'])):
        first_tuples[i][0] = sample_1['pairs'][i][0]
        first_tuples[i][1] = sample_1['pairs'][i][1]
        first_target[i] = 1 
        
    for i in range(len(sample_2['pairs'])):
        first_tuples[i+len(sample_1['pairs'])][0] = sample_1['pairs'][0][0]
        first_tuples[i+len(sample_1['pairs'])][1] = sample_2['pairs'][i][1]
    
    #tuples of second target word 
    for i in range(len(sample_2['pairs'])):
        second_tuples[i][0] = sample_2['pairs'][i][0]
        second_tuples[i][1] = sample_2['pairs'][i][1]
        second_target[i] = 1 
    
    for i in range(len(sample_1['pairs'])):
        second_tuples[i+len(sample_2['pairs'])][0] = sample_2['pairs'][0][0]
        second_tuples[i+len(sample_2['pairs'])][1] = sample_1['pairs'][i][1]
  
    #print(first_target)
    #print(second_target)
    #print(first_tuples)
    #print(second_tuples)
    
    # prediction 
    first_predict_logit = torch.zeros(len(sample_2['pairs'])+len(sample_1['pairs'])).to(device)
    second_predict_logit = torch.zeros(len(sample_2['pairs'])+len(sample_1['pairs'])).to(device)

    
    for i in range(len(sample_2['pairs'])+len(sample_1['pairs'])):
        target_embed, context_embed = model((first_tuples[i][0],first_tuples[i][1]))
        inner_product = torch.dot(torch.flatten(target_embed),torch.flatten(context_embed))
        first_predict_logit[i] = inner_product

    m = nn.Softmax(dim=0).to(device)
    result_first = m(first_predict_logit).to(device)
    loss_first = -torch.sum(first_target * torch.log(result_first)).to(device)


    for i in range(len(sample_2['pairs'])+len(sample_1['pairs'])):
        target_embed, context_embed = model((second_tuples[i][0],second_tuples[i][1]))
        inner_product = torch.dot(torch.flatten(target_embed),torch.flatten(context_embed)).to(device)
        second_predict_logit[i] = inner_product
    
    #print(first_predict_logit)
    #print(second_predict_logit)
    
    result_second = m(second_predict_logit).to(device)
    loss_second = -torch.sum(second_target * torch.log(result_second)).to(device)
    
    #print(result_first)
    #print(result_second)
    
    #print(loss_first)
    #print(loss_second)

    
    total_loss = (loss_first + loss_second).to(device)
    
    print(total_loss)
        
    total_loss.backward()
    # your code here
    optimizer.step()
    return total_loss/2
    # return the current loss value
    ##--------------write above-------------##
    
    ## sample_1 will contain positive (target_1, context_1) tuples
    ## sample_2 will contain another positive (target_2, context_2) tuples
    ## But for the negative sampling, we need negative (target, context) tuples.
    ## NEGATIVE TUPLES can be generated by (target_1, context_2), (target_2, context_1)
    ## (+5) Generate and give positive & negative tuples for model's input
    ##### Then, you would get : target_embed, context_embed = model((target, context)).
    ## (+4) calculate the distance between target_embed and context_embed by DOT PRODUCT
    ## (+4) calculate the loss based on that distance, and optimize the model
    ####    Label positive tuples as class '1', otherwise as class '0'
    ####    You can also use sigmoid function rather than the softmax function.
    ####    At the end, you must return the current loss value.

In [31]:
### DO NOT TOUCH BELOW. JUST USE THESE LINES. MAKE YOUR CODE WORK THESE CODES.
### PENALTY (-5) WILL BE GIVEN WHEN YOUR CODE RAISES AN ERROR DURING EPOCH.
### YOUR TRAINING NEEDS TO BE PERFORMED WITH VARIOUS .TXT FILES.
max_epoch = 1
for epoch in range(max_epoch):
    total_loss = 0.0
    cnt = 0
    for sample in tqdm(dataloader):
        if cnt > 0:
            curr_loss = train(model, optimizer, sample, prev_sample)
            total_loss += curr_loss / len(dataloader)    
        prev_sample = sample
        cnt += 1
        if cnt % 200 == 0:
            print('[EPOCH {}] SAMPLED TRAIN LOSS : {}'.format(epoch, curr_loss))
    print('[EPOCH {}] TOTAL LOSS : {}'.format(epoch, total_loss))

877
1
1096
11
tensor([1, 1, 1, 1, 1, 1, 0, 0, 0, 0], device='cuda:0')
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([[1086, 1449],
        [1086,  819],
        [1086, 1086],
        [1086, 1086],
        [1086, 1883],
        [1086, 1688],
        [1086, 2090],
        [1086, 1377],
        [1086,  983],
        [1086, 1376]], device='cuda:0')
tensor([[2072, 2090],
        [2072, 1377],
        [2072,  983],
        [2072, 1376],
        [2072, 1449],
        [2072,  819],
        [2072, 1086],
        [2072, 1086],
        [2072, 1883],
        [2072, 1688]], device='cuda:0')
tensor([0.2796, 0.1601, 0.2023, 0.2023, 0.1634, 0.2013, 0.1689, 0.1927, 0.1378,
        0.1001], device='cuda:0', grad_fn=<CopySlices>)
tensor([0.1656, 0.1820, 0.1540, 0.1222, 0.2591, 0.1654, 0.1649, 0.1649, 0.1921,
        0.2107], device='cuda:0', grad_fn=<CopySlices>)
tensor([0.1103, 0.0978, 0.1021, 0.1021, 0.0982, 0.1020, 0.0987, 0.1011, 0.0957,
        0.0921], device='cuda:0', grad_fn=<Sof

tensor([1, 1, 1, 0, 0, 0], device='cuda:0')
tensor([1, 1, 1, 0, 0, 0], device='cuda:0')
tensor([[1449, 1467],
        [1449, 1550],
        [1449, 1944],
        [1449,   18],
        [1449, 2124],
        [1449, 1968]], device='cuda:0')
tensor([[1111,   18],
        [1111, 2124],
        [1111, 1968],
        [1111, 1467],
        [1111, 1550],
        [1111, 1944]], device='cuda:0')
tensor([ 0.6504,  0.5756,  0.6814, -0.2290,  0.2035,  0.2337], device='cuda:0',
       grad_fn=<CopySlices>)
tensor([ 0.5051,  0.1538,  0.1073, -0.1108, -0.0612, -0.1185], device='cuda:0',
       grad_fn=<CopySlices>)
tensor([0.2140, 0.1986, 0.2207, 0.0888, 0.1369, 0.1411], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
tensor([0.2488, 0.1751, 0.1671, 0.1344, 0.1412, 0.1334], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
tensor(4.6694, device='cuda:0', grad_fn=<NegBackward>)
tensor(4.9225, device='cuda:0', grad_fn=<NegBackward>)
tensor(9.5919, device='cuda:0', grad_fn=<AddBackward0>)
86
0
tensor(

tensor(7.8149, device='cuda:0', grad_fn=<NegBackward>)
tensor(13.1245, device='cuda:0', grad_fn=<AddBackward0>)
1728
2
tensor([1, 1, 1, 1, 0, 0, 0], device='cuda:0')
tensor([1, 1, 1, 0, 0, 0, 0], device='cuda:0')
tensor([[1628, 1419],
        [1628, 1449],
        [1628,  255],
        [1628,  336],
        [1628, 1449],
        [1628,  950],
        [1628,  720]], device='cuda:0')
tensor([[1377, 1449],
        [1377,  950],
        [1377,  720],
        [1377, 1419],
        [1377, 1449],
        [1377,  255],
        [1377,  336]], device='cuda:0')
tensor([0.1469, 0.2100, 0.1259, 0.1537, 0.2100, 0.1596, 0.1661],
       device='cuda:0', grad_fn=<CopySlices>)
tensor([ 0.8290,  0.2946,  0.2557,  0.1354,  0.8290, -0.0334,  0.1420],
       device='cuda:0', grad_fn=<CopySlices>)
tensor([0.1399, 0.1490, 0.1370, 0.1408, 0.1490, 0.1417, 0.1426],
       device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0.2186, 0.1281, 0.1232, 0.1093, 0.2186, 0.0923, 0.1100],
       device='cuda:0', grad_fn=<

tensor([0.4317, 0.1381, 0.1060, 0.0953, 0.7445], device='cuda:0',
       grad_fn=<CopySlices>)
tensor([0.4880, 0.4407, 0.1710, 0.1380, 0.1078], device='cuda:0',
       grad_fn=<CopySlices>)
tensor([0.2198, 0.1639, 0.1587, 0.1570, 0.3005], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
tensor([0.2457, 0.2343, 0.1789, 0.1731, 0.1680], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
tensor(7.0155, device='cuda:0', grad_fn=<NegBackward>)
tensor(1.4038, device='cuda:0', grad_fn=<NegBackward>)
tensor(8.4193, device='cuda:0', grad_fn=<AddBackward0>)
520
1
tensor([1, 1, 1, 1, 0, 0, 0, 0], device='cuda:0')
tensor([1, 1, 1, 1, 0, 0, 0, 0], device='cuda:0')
tensor([[1578, 1021],
        [1578,  998],
        [1578,  292],
        [1578, 1315],
        [1578,  312],
        [1578,  395],
        [1578, 1849],
        [1578,  407]], device='cuda:0')
tensor([[1376,  312],
        [1376,  395],
        [1376, 1849],
        [1376,  407],
        [1376, 1021],
        [1376,  998],
        [137

tensor([0.1096, 0.1697, 0.1942, 0.1620, 0.1823, 0.1388, 0.1397, 0.1739],
       device='cuda:0', grad_fn=<CopySlices>)
tensor([0.1277, 0.1322, 0.1332, 0.1137, 0.1192, 0.1214, 0.1252, 0.1274],
       device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0.1190, 0.1263, 0.1295, 0.1254, 0.1279, 0.1225, 0.1226, 0.1269],
       device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor(8.2710, device='cuda:0', grad_fn=<NegBackward>)
tensor(8.3187, device='cuda:0', grad_fn=<NegBackward>)
tensor(16.5897, device='cuda:0', grad_fn=<AddBackward0>)
1315
1
tensor([1, 1, 1, 0, 0, 0, 0], device='cuda:0')
tensor([1, 1, 1, 1, 0, 0, 0], device='cuda:0')
tensor([[1287, 1111],
        [1287, 1964],
        [1287, 1985],
        [1287,  464],
        [1287,  871],
        [1287, 1701],
        [1287, 1111]], device='cuda:0')
tensor([[1223,  464],
        [1223,  871],
        [1223, 1701],
        [1223, 1111],
        [1223, 1111],
        [1223, 1964],
        [1223, 1985]], device='cuda:0')
tensor([0.1226, 0.192

...

tensor([1, 1, 1, 0, 0, 0], device='cuda:0')
tensor([1, 1, 1, 0, 0, 0], device='cuda:0')
tensor([[1909,   34],
        [1909, 1376],
        [1909, 1466],
        [1909, 1410],
        [1909, 1111],
        [1909, 1634]], device='cuda:0')
tensor([[1376, 1410],
        [1376, 1111],
        [1376, 1634],
        [1376,   34],
        [1376, 1376],
        [1376, 1466]], device='cuda:0')
tensor([0.7993, 0.6383, 0.9013, 0.5588, 0.4771, 0.3850], device='cuda:0',
       grad_fn=<CopySlices>)
tensor([0.4907, 0.5729, 0.7158, 0.6177, 1.1728, 0.9695], device='cuda:0',
       grad_fn=<CopySlices>)
tensor([0.1949, 0.1659, 0.2159, 0.1533, 0.1412, 0.1288], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
tensor([0.1240, 0.1346, 0.1553, 0.1408, 0.2452, 0.2001], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
tensor(4.9645, device='cuda:0', grad_fn=<NegBackward>)
tensor(5.9556, device='cuda:0', grad_fn=<NegBackward>)
tensor(10.9201, device='cuda:0', grad_fn=<AddBackward0>)
113
5
tensor([1, 1, 1, 

tensor([0.1012, 0.0940, 0.0942, 0.0957, 0.1247, 0.0952, 0.1154, 0.1070, 0.0862,
        0.0862], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor(13.2368, device='cuda:0', grad_fn=<NegBackward>)
tensor(9.3636, device='cuda:0', grad_fn=<NegBackward>)
tensor(22.6003, device='cuda:0', grad_fn=<AddBackward0>)
947
7
tensor([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([[ 480, 1348],
        [ 480, 1348],
        [ 480, 1348],
        [ 480, 1348],
        [ 480, 1348],
        [ 480, 1348],
        [ 480,  496],
        [ 480, 1411],
        [ 480, 1330],
        [ 480,  332],
        [ 480, 1928],
        [ 480, 1928]], device='cuda:0')
tensor([[1330,  496],
        [1330, 1411],
        [1330, 1330],
        [1330,  332],
        [1330, 1928],
        [1330, 1928],
        [1330, 1348],
        [1330, 1348],
        [1330, 1348],
        [1330, 1348],
        [1330, 1348],
        [1330, 1348]], device='cuda:0

        [ 848, 1968]], device='cuda:0')
tensor([[  18, 1988],
        [  18, 1526],
        [  18, 1111],
        [  18, 2124],
        [  18, 1968],
        [  18,  377],
        [  18, 1376],
        [  18, 1137],
        [  18, 1018]], device='cuda:0')
tensor([0.6814, 0.3292, 0.9098, 1.4071, 0.5237, 0.3512, 0.6814, 0.3575, 1.0453],
       device='cuda:0', grad_fn=<CopySlices>)
tensor([0.6246, 0.6064, 0.7767, 0.5228, 1.0841, 0.7754, 0.0534, 0.2642, 0.6812],
       device='cuda:0', grad_fn=<CopySlices>)
tensor([0.1025, 0.0720, 0.1287, 0.2117, 0.0875, 0.0736, 0.1024, 0.0741, 0.1474],
       device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0.1097, 0.1077, 0.1277, 0.0991, 0.1737, 0.1276, 0.0620, 0.0765, 0.1161],
       device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor(8.5117, device='cuda:0', grad_fn=<NegBackward>)
tensor(10.5585, device='cuda:0', grad_fn=<NegBackward>)
tensor(19.0702, device='cuda:0', grad_fn=<AddBackward0>)
1444
4
tensor([1, 1, 1, 1, 1, 1, 0, 0, 0, 0], device='cuda

tensor([1, 1, 1, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([[1111, 1654],
        [1111, 1754],
        [1111, 1165],
        [1111,   18],
        [1111, 1276],
        [1111, 2029],
        [1111, 1922],
        [1111, 1342],
        [1111, 1411]], device='cuda:0')
tensor([[1468, 1922],
        [1468, 1342],
        [1468, 1411],
        [1468, 1654],
        [1468, 1754],
        [1468, 1165],
        [1468,   18],
        [1468, 1276],
        [1468, 2029]], device='cuda:0')
tensor([0.6115, 0.2976, 0.9296, 0.7298, 0.7448, 0.5973, 0.6956, 0.2585, 0.6482],
       device='cuda:0', grad_fn=<CopySlices>)
tensor([ 1.6865,  1.4744,  0.9822,  0.7436,  0.6342,  0.7194, -0.0814,  0.5177,
         0.4737], device='cuda:0', grad_fn=<CopySlices>)
tensor([0.1088, 0.0795, 0.1496, 0.1225, 0.1244, 0.1073, 0.1184, 0.0765, 0.1129],
       device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0.2380, 0.1926, 0.1177, 0.0927, 0.0831, 0.0905, 0.0406, 0.0740, 0.0708],
       device='cuda:0', grad_fn=<Softm

tensor([0.7691, 1.1276, 1.1001, 0.6755, 0.3766, 1.1914, 1.1359, 1.3231, 0.5966],
       device='cuda:0', grad_fn=<CopySlices>)
tensor([0.0710, 0.0716, 0.1506, 0.1160, 0.1150, 0.0873, 0.0933, 0.2081, 0.0871],
       device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0.0912, 0.1306, 0.1270, 0.0831, 0.0616, 0.1392, 0.1317, 0.1588, 0.0768],
       device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor(13.9297, device='cuda:0', grad_fn=<NegBackward>)
tensor(6.4933, device='cuda:0', grad_fn=<NegBackward>)
tensor(20.4230, device='cuda:0', grad_fn=<AddBackward0>)
395
3
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 0, 0, 0, 0], device='cuda:0')
tensor([[1348, 1701],
        [1348,  599],
        [1348, 1376],
        [1348, 1845],
        [1348, 1165],
        [1348,  312],
        [1348,  464],
        [1348,  912],
        [1348,    7],
        [1348, 1639]], device='cuda:0')
tensor([[1449, 1165],
        [1449,  312],
        [1449,  464],
        [1449,  912

In [26]:
len(dataset.trimmed_lyrics[1156])

1

In [ ]:
### MUST TO DO 7 : (+10)
##--------------write below-------------##
# your code here
##--------------write above-------------##
## (+10) bring your word embedding as a numpy array "embedding"
#### "embedding" should be N by D array, 
#### where N is the number of vocabularies, and D is the dimension of the word embedding.
#### embedding[i, :] should be word embedding of dataset.vocabulary[i]

In [ ]:
### JUST USE THESE LINES. MAKE YOUR CODE WORK THESE CODES.
reducer = PCA(n_components=2)
# or try use
# reducer = TSNE(n_components=2, verbose=1)
reduce_results = reducer.fit_transform(embedding)

In [ ]:
### DO NOT TOUCH BELOW. JUST USE THESE LINES. MAKE YOUR CODE WORK THESE CODES.
top_k = 50
sort_idx = np.argsort(list(dataset.frequency.values()))[::-1]
sort_idx = sort_idx[:top_k]
frequent_vocabs = [list(dataset.frequency.keys())[si] for si in sort_idx]
plt.figure(figsize=(10, 6))

for idx, vocab in enumerate(dataset.vocabulary):
    if vocab in frequent_vocabs:
        plt.plot(reduce_results[idx, 0], reduce_results[idx, 1], '.')
        plt.text(reduce_results[idx, 0], reduce_results[idx, 1], vocab)\

In [ ]:
### DO NOT TOUCH BELOW. JUST USE THESE LINES. MAKE YOUR CODE WORK THESE CODES.
min_dist = 10000000
target_word = 'i'
for idx, vocab in enumerate(dataset.vocabulary):
    if vocab != target_word:
        distance = np.linalg.norm(embedding[dataset.vocabulary.index(target_word)] - embedding[idx])
        min_dist = min(distance, min_dist)
        if min_dist == distance:
            nearest_to_target = vocab
print('"{}" is nearest to "{}"'.format(target_word, nearest_to_target))